In [22]:
import pandas_datareader.data as web
import pandas as pd
import datetime 
from threading import Thread
import talib
from heapq import nlargest




class YahooData:
    def __init__(self):
        self.mom63 = {}
        self.mom126 = {}
        self.mom252 = {}
        self.sma = []
        self.symbols = ['BP.L', 'GSK.L', 'BA.L', 'GLEN.L', 'SHEL.L', 'EMG.L', 'CNA.L', 'ABDN.L', 
                'AGR.L', 'AV.L', 'BARC.L', 'BDEV.L', 'BRBY.L', 'BT-A.L', 'CEY.L', 'CINE.L', 'DGE.L', 'EZJ.L',
               'FRAS.L', 'HBR.L', 'HMSO.L', 'HSBA.L', 'IAG.L', 'III.L', 'IWG.L', 'LGEN.L', 'LLOY.L', 'LSEG.L',
               'NWG.L', 'QLT.L', 'RCH.L', 'RIO.L', 'RR.L', 'SBRY.L', 'SPI.L', 'STAN.L', 'TLW.L', 'TSCO.L', 'ULVR.L', 'VOD.L',
               'AIR.PA', 'AF.PA', 'AI.PA', 'CS.PA', 'BNP.PA', 'BOL.PA', 'EN.PA', 'ACA.PA', 'CA.PA', 'BN.PA', 'ENGI.PA',
               'MC.PA', 'ORA.PA', 'OR.PA', 'RNO.PA', 'GLE.PA', 'STM.PA', 'ADS.DE', 'ALV.DE', 'BAS.DE', 'BMW.DE', 'BOSS.DE',
               'CBK.DE', 'DBK.DE', 'DPW.DE', 'DTE.DE', 'EOAN.DE', 'LHA.DE', 'LIN.DE', 'MBG.DE', 'PUM.DE', 'SAP.DE', 'SIE.DE',
               'TUI1.DE', 'VNA.DE', 'VOW3.DE', 'MMM', 'AIG', 'T', 'ANF', 'ADBE', 'AA', 'BABA', 'MO', 'AMZN', 'AXP', 'AAPL', 'BIDU', 'BA', 'BAC', 'BMY', 'CAT', 'CVX',
         'CSCO', 'C', 'KO', 'CMCSA', 'COP', 'COST', 'DIS', 'EBAY', 'XOM', 'FB', 'FDX', 'RACE', 'F', 'GE', 'GM', 'GS', 'GOOG',
         'HPE', 'HAL', 'HOG', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'LVS', 'MS', 'MGM', 'MA', 'MCD', 'MSFT', 'MCO',
         'MSI', 'NFLX', 'NKE', 'NVS', 'NVDA', 'ORCL', 'PG', 'PZZA', 'PEP', 'PBR', 'PFE', 'PM', 'QCOM', 'SCHW', 'SHAK',
         'SBUX', 'TSLA', 'TM', 'TRIP', 'TWTR', 'UPS', 'VALE', 'VZ', 'V', 'WMT', 'WFC', 'YNDX', 'YUM']
    
    def get_data(self, symbol):
        df = web.DataReader(symbol, 'yahoo', start='2021-03-23', end='2022-05-30')
        close = df['Adj Close']
        roc63 = talib.ROCP(close, timeperiod=63)
        roc126 = talib.ROCP(close, timeperiod=126)
        roc252 = talib.ROCP(close, timeperiod=252)
        sma50 = talib.SMA(close, 50)
        sma100 = talib.SMA(close, 100)
        sma200 = talib.SMA(close, 200)
        self.mom63[symbol] = roc63[-1]
        self.mom126[symbol] = roc126[-1]
        self.mom252[symbol] = roc252[-1]
        if sma50[-1] > sma100[-1] > sma200[-1]:
            self.sma.append(symbol)

    
    def get_results(self):
        threads = []
        for symbol in self.symbols:
            threads.append(Thread(target=self.get_data, args=[symbol]))
            
        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()
        
        top_ten63 = nlargest(10, self.mom63, key = self.mom63.get) 
        top_ten126 = nlargest(10, self.mom126, key = self.mom126.get) 
        top_ten252 = nlargest(10, self.mom252, key = self.mom252.get) 
        top_ten_all = [*top_ten63, *top_ten126, *top_ten252, *self.sma]
        return set(top_ten_all)

In [23]:
data = YahooData()
result = data.get_results()
result

{'AA',
 'AI.PA',
 'AIG',
 'BA.L',
 'BMY',
 'BP.L',
 'CA.PA',
 'CAT',
 'CNA.L',
 'COP',
 'COST',
 'CVX',
 'DGE.L',
 'DTE.DE',
 'EMG.L',
 'GLEN.L',
 'GSK.L',
 'HAL',
 'HBR.L',
 'IBM',
 'JNJ',
 'KHC',
 'KO',
 'LHA.DE',
 'LIN.DE',
 'LSEG.L',
 'MO',
 'NVS',
 'ORA.PA',
 'PBR',
 'PEP',
 'PFE',
 'RIO.L',
 'SHEL.L',
 'STAN.L',
 'T',
 'TLW.L',
 'VALE',
 'XOM'}